In [327]:
import pandas as pd
import numpy as np

df = pd.read_excel('MR.xlsx')

df

,vessel_id,safety_score,dwt,main_engine_fuel_type,adjusted_monthly_cost,total_co2_eq,total_fuel_con_tonnes_meAeAb
0,10007920,1,171810,DISTILLATE FUEL,2.684675e+06,7091.811598,2175.689421
1,10015840,1,49949,DISTILLATE FUEL,6.725528e+05,876.425795,269.080025
2,10023760,1,209878,DISTILLATE FUEL,9.303178e+05,719.556518,220.924516
3,10031680,1,179134,DISTILLATE FUEL,7.631470e+05,112.665207,34.611903
4,10039600,2,49999,DISTILLATE FUEL,6.020438e+05,724.354289,222.474217
...,...,...,...,...,...,...,...
103,10823580,3,39538,LNG,4.394938e+05,249.502804,85.345440
104,10831500,4,208947,LNG,1.055960e+06,468.585675,160.710690
105,10839410,3,208970,LNG,1.089991e+06,500.500070,173.173807
106,10847330,5,208808,LNG,1.019638e+06,446.620288,154.795526


In [320]:
df['price_per_dwt'] = df['adjusted_monthly_cost']/df['dwt']
df['price_per_dwt_rank'] = df['price_per_dwt'].rank()
df['safety_score_rank'] = df['safety_score'].rank(ascending=False)
df['overall_score'] = df['price_per_dwt_rank']+df['safety_score_rank']

maX safety and min cost

In [ ]:
best_fleet = pd.DataFrame()

fuel_type = ['Methanol', 'DISTILLATE FUEL', 'LNG', 'Ammonia', 'Ethanol', 'LPG (Butane)', 'Hydrogen', 'LPG (Propane)']

for type in fuel_type:
    check = df[df['main_engine_fuel_type']==type]
    best = check[check['overall_score'] == check['overall_score'].min()]
    best_fleet = pd.concat([best_fleet,best], ignore_index=True)

In [143]:
dwt_crit = 54.92/12*1000000
curr_dwt = best_fleet['dwt'].sum()
filter_df = df

while curr_dwt<dwt_crit:
    next_best = filter_df[filter_df['overall_score'] == filter_df['overall_score'].min()]
    best_fleet = pd.concat([best_fleet,next_best], ignore_index=True,verify_integrity=True)
    filter_df.drop(next_best.index,inplace=True)
    curr_dwt = best_fleet['dwt'].sum()


min cost

In [ ]:
def safety_weights(row):
    safety_rank = row['safety_score_rank']
    if safety_rank<2:
        return safety_rank+1
    return safety_rank

def price_weights(row):
    safety_rank = row['price_per_dwt_rank']
    if safety_rank<2:
        return safety_rank+1
    return safety_rank


In [321]:
df['price_per_dwt'] = df['adjusted_monthly_cost']/df['dwt']

df['price_per_dwt_rank'] = df['price_per_dwt'].rank()
df['safety_score_rank'] = df['safety_score'].rank(ascending=False)
df['overall_score'] = df['price_per_dwt_rank']*2+df['safety_score_rank']


best_fleet = pd.DataFrame()

fuel_type = ['Methanol', 'DISTILLATE FUEL', 'LNG', 'Ammonia', 'Ethanol', 'LPG (Butane)', 'Hydrogen', 'LPG (Propane)']

for type in fuel_type:
    check = df[df['main_engine_fuel_type']==type]
    best = check[check['price_per_dwt'] == check['price_per_dwt'].min()]
    best_fleet = pd.concat([best_fleet,best])

dwt_crit = 54.92/12*1000000
curr_dwt = best_fleet['dwt'].sum()
filter_df = df.copy()

while curr_dwt<dwt_crit:
    next_best = filter_df[filter_df['price_per_dwt'] == filter_df['price_per_dwt'].min()]
    try:
        best_fleet = pd.concat([best_fleet,next_best],verify_integrity=True)
    except ValueError:
        filter_df.drop(next_best.index,inplace=True)
    curr_dwt = best_fleet['dwt'].sum()


In [322]:
best_fleet['adjusted_monthly_cost'].sum()

20679705.531387106

In [323]:
best_fleet['safety_score'].mean()

3.1818181818181817

In [324]:
best_fleet['vessel_id'].nunique()

22

In [325]:
best_fleet

,vessel_id,safety_score,dwt,main_engine_fuel_type,adjusted_monthly_cost,total_co2_eq,total_fuel_con_tonnes_meAeAb,price_per_dwt,price_per_dwt_rank,safety_score_rank,overall_score
69,10554330,4,210102,Methanol,1.249846e+06,578.477914,357.058627,5.948760,40.0,32.0,112.0
18,10150460,1,262781,DISTILLATE FUEL,9.113014e+05,658.254483,198.616045,3.467912,1.0,100.0,102.0
97,10776060,4,261089,LNG,1.107065e+06,621.837009,215.602865,4.240183,12.0,32.0,56.0
80,10641440,5,209485,Ammonia,1.143756e+06,153.960416,360.084056,5.459848,33.0,9.0,75.0
72,10578090,4,207939,Ethanol,1.095734e+06,499.555025,221.085714,5.269500,32.0,32.0,96.0
57,10459300,4,210909,LPG (Butane),1.067633e+06,650.286390,209.219062,5.062057,26.0,32.0,84.0
84,10673120,3,178838,Hydrogen,1.190011e+06,103.666831,103.837717,6.654129,48.0,62.0,158.0
55,10443460,5,205905,LPG (Propane),1.031941e+06,739.099629,240.068820,5.011735,24.0,9.0,57.0
10,10087110,2,261085,DISTILLATE FUEL,9.355641e+05,914.881110,274.327578,3.583370,2.0,84.5,88.5
29,10237570,1,261046,DISTILLATE FUEL,9.467520e+05,778.900914,239.286324,3.626763,3.0,100.0,106.0


4.0

In [317]:
df['price_per_dwt'] = df['adjusted_monthly_cost']/df['dwt']

df['price_per_dwt_rank'] = df['price_per_dwt'].rank()
df['safety_score_rank'] = df['safety_score'].rank(ascending=False)
df['overall_score'] = df['price_per_dwt_rank']*2+df['safety_score_rank']

In [315]:
def derive_multiplier(m_max=2,m_min=1,increment=0.1, safety_avg = 4,dwt_crit = 54.92/12*1000000):

    diff_from_safetyavg = 100
    min_cost = float("inf")
    price_multiplier = m_max
    while price_multiplier>m_min:
        filter_df = df.copy()
        filter_df['overall_score'] = filter_df['price_per_dwt_rank']*price_multiplier+filter_df['safety_score_rank']
        best_fleet = pd.DataFrame()
        fuel_type = ['Methanol', 'DISTILLATE FUEL', 'LNG', 'Ammonia', 'Ethanol', 'LPG (Butane)', 'Hydrogen', 'LPG (Propane)']
        for type in fuel_type:
            check = filter_df[filter_df['main_engine_fuel_type']==type]
            best = check[check['overall_score'] == check['overall_score'].min()]
            best_fleet = pd.concat([best_fleet,best], ignore_index=True)
            filter_df.drop(best.index,inplace=True)

        curr_dwt = best_fleet['dwt'].sum()
        while curr_dwt<dwt_crit:
            next_best = filter_df[filter_df['overall_score'] == filter_df['overall_score'].min()]
            try:
                best_fleet = pd.concat([best_fleet,next_best],verify_integrity=True)
            except ValueError:
                filter_df.drop(next_best.index,inplace=True)
            curr_dwt = best_fleet['dwt'].sum()
        
        avg = best_fleet['safety_score'].mean()
        cost = best_fleet['adjusted_monthly_cost'].sum()
        # print(best_fleet,avg,cost)
        if (avg >= safety_avg) and (avg-safety_avg <= diff_from_safetyavg) and (cost<=min_cost):
            optimal_multipler = price_multiplier
            diff_from_safetyavg = avg-safety_avg
            min_cost = cost
            optimal_fleet = best_fleet

        price_multiplier -= increment

    return optimal_fleet,optimal_multipler, safety_avg+diff_from_safetyavg,min_cost
    


In [326]:
derive_multiplier(m_max=5,m_min=1,increment=0.1, safety_avg = 4)

(     vessel_id  safety_score     dwt main_engine_fuel_type  \
 0     10554330             4  210102              Methanol   
 1     10332600             5  207999       DISTILLATE FUEL   
 2     10776060             4  261089                   LNG   
 3     10641440             5  209485               Ammonia   
 4     10578090             4  207939               Ethanol   
 5     10459300             4  210909          LPG (Butane)   
 6     10673120             3  178838              Hydrogen   
 7     10443460             5  205905         LPG (Propane)   
 30    10245490             5  210938       DISTILLATE FUEL   
 16    10134620             5  209997       DISTILLATE FUEL   
 50    10403870             4  210177       DISTILLATE FUEL   
 53    10427630             4  207999       DISTILLATE FUEL   
 106   10847330             5  208808                   LNG   
 52    10419710             4  180391       DISTILLATE FUEL   
 42    10340520             3  209529       DISTILLATE 

In [328]:
df = pd.read_excel('MR.xlsx')

df['price_per_dwt'] = df['adjusted_monthly_cost']/df['dwt']

df['price_per_dwt_rank'] = df['price_per_dwt'].rank()
df['safety_score_rank'] = df['safety_score'].rank(ascending=False)
df['overall_score'] = df['price_per_dwt_rank']*2.4+df['safety_score_rank']


best_fleet = pd.DataFrame()

fuel_type = ['Methanol', 'DISTILLATE FUEL', 'LNG', 'Ammonia', 'Ethanol', 'LPG (Butane)', 'Hydrogen', 'LPG (Propane)']

for type in fuel_type:
    check = df[df['main_engine_fuel_type']==type]
    best = check[check['overall_score'] == check['overall_score'].min()]
    best_fleet = pd.concat([best_fleet,best])

dwt_crit = 54.92/12*1000000
curr_dwt = best_fleet['dwt'].sum()
filter_df = df.copy()

while curr_dwt<dwt_crit:
    next_best = filter_df[filter_df['overall_score'] == filter_df['overall_score'].min()]
    try:
        best_fleet = pd.concat([best_fleet,next_best],verify_integrity=True)
    except ValueError:
        filter_df.drop(next_best.index,inplace=True)
    curr_dwt = best_fleet['dwt'].sum()

In [329]:
best_fleet['adjusted_monthly_cost'].sum(),best_fleet['safety_score'].mean()

(21741102.346712016, 4.090909090909091)

In [338]:
best_fleet['main_engine_fuel_type'].nunique()

8